## Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# Scrap the data from Wiki and Form the Table
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(wiki).text

In [3]:
soup = BeautifulSoup(source, 'xml')


In [4]:
table=soup.find('table')

In [5]:
column_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [6]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
#Drop the 'Not assigned' from Borough
df=df[df['Borough']!='Not assigned']

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [10]:
df.shape

(103, 3)

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [13]:
# Neighborhoods separated with a comma 
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [16]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df3 = pd.merge(df, df2, left_on='PostalCode', right_on='Postal Code')

In [18]:
df3=df3.drop(columns='Postal Code')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [19]:
df3.shape

(103, 5)

# Part 3

## Necessery libraries

In [22]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# install folium
!conda install -c conda-forge folium=0.5.0 --yes #
import folium # map rendering library

# install geopy
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

## The map of Toronto with neighborhoods 

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [48]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## The map of Toronto with neighborhoods whose name contain 'Toronto'

In [36]:
tor_data = df3[df3['Borough'].str.contains('Toronto') != False].reset_index(drop=True)
tor_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [49]:
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough'], tor_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

## Foursquare Credentials and Version

In [50]:
CLIENT_ID = 'TB4AM3O1NNWBZ5U2NZABHNO1R3NTHMFLXL3PNTMEQON4Y2FU' 
CLIENT_SECRET = 'MXMYN4KVSXXQYUYU32JZNLMMGGLWLZEV41THWOE1Z312CMKJ' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TB4AM3O1NNWBZ5U2NZABHNO1R3NTHMFLXL3PNTMEQON4Y2FU
CLIENT_SECRET:MXMYN4KVSXXQYUYU32JZNLMMGGLWLZEV41THWOE1Z312CMKJ


## Restaurants in each Borough

In [123]:
# the function to find Restaurants
def getNearbyrestaurant(names, latitudes, longitudes, search_query='Restaurant', LIMIT=200, radius=500):
    
    restaurant_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            search_query,
            radius, 
            LIMIT)
            
        # the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        restaurant_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_restaurant = pd.DataFrame([item for restaurant_list in restaurant_list for item in restaurant_list])
    nearby_restaurant.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_restaurant)


In [117]:
tor_restaurants = getNearbyrestaurant(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst
 Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

### Restaurants in tor_restaurants dataframe

In [118]:
print(tor_restaurants.shape)
tor_restaurants.head()

(1381, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
1,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
2,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Brick Street Bakery,43.650574,-79.359539,Bakery
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [145]:
tor_restaurants.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,52,52,52,52,52,52
"Brockton, Parkdale Village, Exhibition Place",15,15,15,15,15,15
Business reply mail Processing CentrE,5,5,5,5,5,5
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst\n Quay, South Niagara, Island airport",2,2,2,2,2,2
Central Bay Street,70,70,70,70,70,70
Christie,7,7,7,7,7,7
Church and Wellesley,62,62,62,62,62,62
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,31,31,31,31,31,31


## Restaurant categories in Toronto

In [122]:
print('There are {} unique types restaurants in Toronto.'.format(len(tor_restaurants['Venue Category'].unique())))

There are 90 unique types restaurants in Toronto.


In [141]:
tor_restaurants_cat = pd.get_dummies(tor_restaurants[['Venue Category']], prefix="", prefix_sep="")

tor_restaurants_cat.insert(0, 'Neighborhood', tor_restaurants['Neighborhood']) 

tor_restaurants_cat.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
tor_restaurants_cat_grouped = tor_restaurants_cat.groupby('Neighborhood').sum().reset_index()
tor_restaurants_cat_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Berczy Park,0,1,0,0,0,1,1,4,1,...,1,2,0,0,1,1,0,2,0,0
1,"Brockton, Parkdale Village, Exhibition Place",0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,0
2,Business reply mail Processing CentrE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Central Bay Street,0,1,0,0,0,0,0,0,0,...,1,3,0,0,0,3,0,1,0,0


## Top-5 Restaurant categories in Each Neighborhood in Toronto

In [215]:
num_top_restraunt_type = 5

for hood in tor_restaurants_cat_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_restaurants_cat_grouped[tor_restaurants_cat_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Type of Restaurant','Number of Restaurants']
    temp = temp.iloc[1:]
    temp['Number of Restaurants'] = temp['Number of Restaurants'].astype(int)
    temp = temp.round({'Number of Restaurants': 2})
    print(temp.sort_values('Number of Restaurants', ascending=False).reset_index(drop=True).head(num_top_restraunt_type))
    print('\n')

----Berczy Park----
   Type of Restaurant  Number of Restaurants
0          Restaurant                      7
1  Italian Restaurant                      4
2              Bakery                      4
3              Bistro                      2
4   French Restaurant                      2


----Brockton, Parkdale Village, Exhibition Place----
  Type of Restaurant  Number of Restaurants
0               Café                      4
1         Restaurant                      2
2             Bakery                      2
3     Breakfast Spot                      2
4     Sandwich Place                      1


----Business reply mail Processing CentrE----
     Type of Restaurant  Number of Restaurants
0  Fast Food Restaurant                      2
1           Pizza Place                      1
2            Restaurant                      1
3         Burrito Place                      1
4     Afghan Restaurant                      0


----CN Tower, King and Spadina, Railway Lands, Harbourfront

In [158]:
import numpy as np

In [197]:
def return_most_common_restaurants(row, num_top_restraunt_type):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_restraunt_type]

In [232]:
num_top_restraunt_type = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_restraunt_type):
    try:
        columns.append('{}{} Most Common Restraunt Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restraunt Type'.format(ind+1))

# create a new dataframe
neighborhoods_rest_sorted = pd.DataFrame(columns=columns)
neighborhoods_rest_sorted['Neighborhood'] = tor_restaurants_cat_grouped['Neighborhood']

for ind in np.arange(tor_restaurants_cat_grouped.shape[0]):
    neighborhoods_rest_sorted.iloc[ind, 1:] = return_most_common_restaurants(tor_restaurants_cat_grouped.iloc[ind, :], num_top_restraunt_type)

neighborhoods_rest_sorted.head()

,Neighborhood,1st Most Common Restraunt Type,2nd Most Common Restraunt Type,3rd Most Common Restraunt Type,4th Most Common Restraunt Type,5th Most Common Restraunt Type
0,Berczy Park,Restaurant,Italian Restaurant,Bakery,French Restaurant,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Restaurant,Sandwich Place
2,Business reply mail Processing CentrE,Fast Food Restaurant,Restaurant,Burrito Place,Pizza Place,Wings Joint
3,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Tapas Restaurant,Wings Joint,Falafel Restaurant,Dim Sum Restaurant
4,Central Bay Street,Café,Restaurant,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant


## Clustering the Neighborhoods (n_clusters=5)

In [200]:
kclusters = 5

tor_restaurants_cat_grouped_clustering = tor_restaurants_cat_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_restaurants_cat_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 2, 2, 2, 4, 2, 0, 3, 0, 2], dtype=int32)

## Adding Cluster labels and finalizing the dataframe 

In [233]:
neighborhoods_rest_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [234]:
neighborhoods_rest_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Restraunt Type,2nd Most Common Restraunt Type,3rd Most Common Restraunt Type,4th Most Common Restraunt Type,5th Most Common Restraunt Type
0,1,Berczy Park,Restaurant,Italian Restaurant,Bakery,French Restaurant,Diner
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Restaurant,Sandwich Place
2,2,Business reply mail Processing CentrE,Fast Food Restaurant,Restaurant,Burrito Place,Pizza Place,Wings Joint
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Tapas Restaurant,Wings Joint,Falafel Restaurant,Dim Sum Restaurant
4,4,Central Bay Street,Café,Restaurant,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant


In [237]:
neighborhoods_rest_cluster_lat_long = neighborhoods_rest_sorted

neighborhoods_rest_cluster_lat_long = neighborhoods_rest_cluster_lat_long.join(tor_data.set_index('Neighborhood'), on='Neighborhood')

In [238]:
neighborhoods_rest_cluster_lat_long.head()

,Cluster Labels,Neighborhood,1st Most Common Restraunt Type,2nd Most Common Restraunt Type,3rd Most Common Restraunt Type,4th Most Common Restraunt Type,5th Most Common Restraunt Type,PostalCode,Borough,Latitude,Longitude
0,1,Berczy Park,Restaurant,Italian Restaurant,Bakery,French Restaurant,Diner,M5E,Downtown Toronto,43.644771,-79.373306
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Restaurant,Sandwich Place,M6K,West Toronto,43.636847,-79.428191
2,2,Business reply mail Processing CentrE,Fast Food Restaurant,Restaurant,Burrito Place,Pizza Place,Wings Joint,M7Y,East Toronto,43.662744,-79.321558
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Tapas Restaurant,Wings Joint,Falafel Restaurant,Dim Sum Restaurant,M5V,Downtown Toronto,43.628947,-79.394420
4,4,Central Bay Street,Café,Restaurant,Italian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,M5G,Downtown Toronto,43.657952,-79.387383


## Visualization of clustered neighborhoods

In [241]:
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_rest_cluster_lat_long['Latitude'], neighborhoods_rest_cluster_lat_long['Longitude'], neighborhoods_rest_cluster_lat_long['Neighborhood'], neighborhoods_rest_cluster_lat_long['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2